# You're probably declaring your python functions the wrong way

A function's signature is the definition of a function input and output, its arguments names and types, if it returns something, and what type is it. Python lets you make several kinds of signatures, from the most generic, to the most specific. In the next sections, we will discuss those signatures, focusing on:

- Code legibility;
- IDE's code inspection features;
- Function reusability;
- Effort to change/maintain code.

## 1. Generic signature - implicit generic arguments, without return type

```python
def to_string(*args, **kwargs):
    ...
```

This kind of signature doesn't give us any clue about what arguments the function is expecting, or what it returns. It's the most generic function we can write because we're not limiting anything. We should only write functions like this if we manage to accept any kind of argument, and make sure it will always execute properly - and this can get pretty hard.

Functions like this can be called with both positional and named arguments, where `args` will be a `tuple` containing positional arguments with preserved order, and `kwargs` will be a `dict` containing named arguments and their values.

In [1]:
def to_string(*args, **kwargs):
    print("args =", args)
    print("kwargs =", kwargs)

to_string(1, True, number=4)

args = (1, True)
kwargs = {'number': 4}


Despite all generalisation we can get when declaring functions like this, unfortunately we often see implementations like below, where the function has generic signature but uses specific arguments to get the job done:

```python
def divide_by(*args, **kwargs):
    return kwargs.get("number") / kwargs.get('by')
```

- **Code legibility:** Low. We only know that there's a function named `divide_by` but we don't know how to call it or what it returns. We need to read its implementation.
- **IDE's code inspection features:**
    - Autocomplete arguments: `No`
    - Missing/Unexpected arguments checking: `No`
    - Arguments typechecking: `No`
- **Function reusability:** `Low` Every time we want to reuse this function we have to remember how to call it and what it returns by reading its implementation.
- **Effort to change/maintain code:** `High` If one accidentally or intentionally changes the argument name from `number` to `quantity`, we'll have to go through all places where `divide_by` is called and correct the argument name too. IDE won't highlight `number` as an unexpected argument.

Copy and paste the snippet below on your favorite IDE and check its autocompletion/highlighting features.

In [2]:
def divide_by(*args, **kwargs):
    return kwargs.get("number") / kwargs.get('by')

def divide_by_safe(*args, **kwargs):
    try:
        print(divide_by(*args, **kwargs))
    except (TypeError, ValueError) as err:
        print(repr(err))


divide_by_safe("Hello")
divide_by_safe(quantity=3)
divide_by_safe(5, 4)
divide_by_safe(number=5, by="a")
divide_by_safe(number=5, by=4)

TypeError("unsupported operand type(s) for /: 'NoneType' and 'NoneType'")
TypeError("unsupported operand type(s) for /: 'NoneType' and 'NoneType'")
TypeError("unsupported operand type(s) for /: 'NoneType' and 'NoneType'")
TypeError("unsupported operand type(s) for /: 'int' and 'str'")
1.25


### 1.1 Generic signature - only positional or named arguments

The first limit we can impose to these functions is to specify whether it will take only positional or named arguments. As it explicitly limits how function can be called, unexpected arguments are caught by IDE's code inspection, and it highlights them with `WARNING` level. However, it doesn't improve any of the points listed above.

```python
# positional only
def to_string(*args):
    ...
```

```python
# named only
def to_string(**kwargs):
    ...
```

Copy and paste the snippets below on your favorite IDE and check its autocompletion/highlighting features.

In [3]:
# only positional
def to_string(*args):
    print("args =", args)

to_string(1, True, 3.4, ["a"], "b")
try:
    to_string(1, True, 3.4, ["a"], "b", named=True)  # highlights named=True, and raises TypeError
except TypeError as err:
    print(repr(err))

args = (1, True, 3.4, ['a'], 'b')
TypeError("to_string() got an unexpected keyword argument 'named'")


In [4]:
# only named
def to_string(**kwargs):
    print("kwargs =", kwargs)

to_string(text="Ipsum lorum", number=3, is_test=False)
try:
    to_string(True, text="Ipsum lorum", number=3, is_test=False) # highlights True, and raises TypeError
except TypeError as err:
    print(repr(err))

kwargs = {'text': 'Ipsum lorum', 'number': 3, 'is_test': False}
TypeError('to_string() takes 0 positional arguments but 1 was given')


## 2. Specific signature - explicit named arguments

The first improvement we apply to our function is to explicitly name the arguments it will receive. It helps not only the reader, but IDE can now autocomplete and check missing/unexpected arguments.

```python
def divide_by(number, by):
    ...
```

- **Code legibility:** Medium. Now we know that `divide_by` receives `number` and `by` arguments, but we don't know what is their type or what the function returns.
- **IDE's code inspection features:**
    - Autocomplete arguments: `Yes`
    - Missing/Unexpected arguments checking: `Yes`
    - Arguments typechecking: `No`
- **Function reusability:** `Medium`. To reuse it we will have to read the implementation to infer the arguments types.
- **Effort to change/maintain code:** High. If one accidentally or intentionally changes the argument name from `number` to `quantity`, we'll have to go through all places where `divide_by` is called and correct the argument name too. IDE won't highlight `number` as an unexpected argument.

### 2.1 Specific signature - typed arguments and return value

```python
def to_string(number: int) -> str:
    ...
```

---
##  Closing thoughts

Characteristics of a well-defined function:

- Function's name should be a verb, or represent an action: this improves code legibility;
- Function's arguments must have explicit types: this makes Pycharm highlight if you are passing the wrong type;
- Function's arguments must have explicit names: avoid using arguments names like `aux`, `data`, `response`. You must be able to tell what is inside the variable only by reading its name;
- Function's arguments must have explicit content: avoid passing dictionaries as arguments because Pycharm can't autocomplete the content, or typecheck values, and afterwards the code maintainer will never know what's inside that dictionary;
- Function must have explicit return type;
- Function's return value must have explicit content: avoid returning dictionaries because Pycharm can't autocomplete the content, or typecheck values, and afterwards the code maintainer will never know what's inside the returned variable.